In [57]:
# Imports here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import cv2
import glob

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [2]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
batch_size = 16
train_path = '../../data/age/train'
val_path = '../../data/age/val'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [58]:
# Define transforms for the training, validation, and validation sets
training_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.631, 0.484, 0.413], 
        [0.216, 0.1940, 0.185]
    )
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.631, 0.484, 0.413], 
        [0.216, 0.1940, 0.185]
    )
])

# TODO: Load the datasets with ImageFolder
training_dataset = datasets.ImageFolder(train_path, transform=training_transforms)
val_dataset = datasets.ImageFolder(val_path, transform=val_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

img_train_count = len(glob.glob(train_path + '/**/*.jpg'))
img_val_count = len(glob.glob(val_path + '/**/*.jpg'))
print(f'train: {img_train_cout}, test: {img_val_count}')

train: 704, test: 54


In [45]:
model = models.resnet18(pretrained=True)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [46]:
# Freeze pretrained model parameters to avoid backpropogating through them
for parameter in model.parameters():
    parameter.requires_grad = True

from collections import OrderedDict

# Build custom classifier
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(25088, 5000)),
    ('relu', nn.ReLU()),
    ('drop', nn.Dropout(p=0.5)),
    ('fc2', nn.Linear(5000, 10)),
    ('output', nn.LogSoftmax(dim=1))
]))

model.classifier = classifier

In [47]:
# Loss function and gradient descent
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [63]:
def exportModel(model, model_name = "last.pt"):
    model_scripted = torch.jit.script(self)
    model_scripted.save(model_name)

In [64]:
epochs = 3

for epoch in range(0, epochs):
    # ---------------------- TRAINING ---------------------------
    print(f"==> Training epoch {epoch}/{epochs - 1}:")

    model.train()
    
    train_accuracy = 0.0
    train_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        # zero out the gradients at the start of a new batch
        optimizer.zero_grad()

        # detect
        outputs = model(images)

        # calculate loss
        loss = loss_function(outputs, labels)

        # back propagation
        loss.backward()

        # update weights and bias
        optimizer.step()

        train_loss += loss.cpu().data * images.size(0)
        _, prediction = torch.max(outputs.data, 1)

        # sum accuracy
        train_accuracy += int(torch.sum(prediction == labels.data))

    # calculate accuracy and loss
    train_accuracy = float(train_accuracy / img_train_count)
    train_loss = float(train_loss / img_train_count)

    # ----------------------- EVALUATION ------------------------
    print(f"\tEvaluating:")

    model.eval()

    val_accuracy = 0.0
    for i, (images, labels) in enumerate(val_loader):
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        # get prediction
        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        val_accuracy += int(torch.sum(prediction == labels.data))

    val_accuracy = val_accuracy / img_val_count

    # --------------------- MANAGE RESULTS ----------------------
    print(f"\t---> train_loss: {train_loss:.3f}")
    print(f"\t---> train_acc:  {train_accuracy:.3f}")
    print(f"\t---> val_acc:   {val_accuracy:.3f}")

    # save the best model
    if export_model and val_accuracy > best_accuracy:
        print("==> Saving the model...")

        exportModel(model)
        best_accuracy = val_accuracy

==> Training epoch 0/2:


KeyboardInterrupt: 

In [ ]:
def test_accuracy(model, test_loader):

    # Do validation on the test set
    model.eval()
    model.to('cuda')

    with torch.no_grad():
    
        accuracy = 0
    
        for images, labels in iter(test_loader):
            if torch.cuda.is_available():
                images = images.cuda()
                labels = labels.cuda()
    
            output = model.forward(images)
            probabilities = torch.exp(output)
            equality = (labels.data == probabilities.max(dim=1)[1])
            accuracy += equality.type(torch.FloatTensor).mean()
        
        print("Test Accuracy: {}".format(accuracy/len(test_loader)))    
        
        
test_accuracy(model, test_loader)